In [1]:
import sys
sys.path.append('..')
print(sys.path)

['c:\\Users\\Vinícius de Lima\\Projects\\covid19datascience\\notebooks', 'c:\\Users\\Vinícius de Lima\\.vscode\\extensions\\ms-toolsai.jupyter-2021.3.684299474\\pythonFiles', 'c:\\Users\\Vinícius de Lima\\.vscode\\extensions\\ms-toolsai.jupyter-2021.3.684299474\\pythonFiles\\lib\\python', 'C:\\ProgramData\\Anaconda3\\python38.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', '', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Vinícius de Lima\\.ipython', '..']


In [2]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta, date

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf, normalize_ibge

import gc

pd.set_option('display.max_columns', None)

output = join("../output","correcoes","tb_pacientes")
input_files = "../input"

if not isdir(output):
    makedirs(output)

regionais = static.regionais.copy()
timer = Timer()

In [3]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()
tb.shape()

(833786, 16124)

In [4]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()
casos_confirmados.shape()

Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0
C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
Casos confirmados excluidos: 0
Obitos confirmados excluidos: 0


(833786, 16124)

In [5]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

casos casos_confirmados: (833786, 24)
casos tb_pacientes: (833786, 48)


In [6]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [7]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [8]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)
if (len(casos_confirmados_nao_tb_pacientes) > 0) or (len(tb_pacientes_nao_casos_confirmados) > 0):
    raise Exception("diferentes")

casos_confirmados_nao_tb_pacientes: (0, 24)
tb_pacientes_nao_casos_confirmados: (0, 48)


In [9]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [10]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [11]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))

False
False
False


In [12]:
timer.reset()

if False:
    sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
    sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
    i = 0
    for (ibge_cc, df_ibge_cc), (ibge_tb, df_ibge_tb) in zip(casosc.groupby('ibge7'),casost.groupby('ibge_res_pr')):
        print(i,end=' ')
        i+=1
        if ibge_cc != ibge_tb:
            raise Exception(ibge_cc,ibge_tb)

        df_ibge_cc = df_ibge_cc.sort_values('nome_hash')
        df_ibge_tb = df_ibge_tb.sort_values('nome_hash')

        duplicados_ibge_cc = df_ibge_cc.loc[ df_ibge_cc.duplicated('nome_hash', keep=False) ]
        duplicados_ibge_tb = df_ibge_tb.loc[ df_ibge_tb.duplicated('nome_hash', keep=False) ]

        sorted_casosc = sorted_casosc.append(df_ibge_cc.loc[~df_ibge_cc.index.isin(duplicados_ibge_cc.index)])
        sorted_casost = sorted_casost.append(df_ibge_tb.loc[~df_ibge_tb.index.isin(duplicados_ibge_tb.index)])

        for (nome_cc, df_nome_cc), (nome_tb, df_nome_tb) in zip(duplicados_ibge_cc.groupby('nome_hash'),duplicados_ibge_tb.groupby('nome_hash')):
            if nome_cc != nome_tb:
                raise Exception(nome_cc,nome_tb)

            sorted_casosc = sorted_casosc.append(df_nome_cc.sort_values('idade'))
            sorted_casost = sorted_casost.append(df_nome_tb.sort_values('idade'))


    print(f"casos casos_confirmados: {sorted_casosc.shape}")
    print(f"casos tb_pacientes: {casost.shape}")
    print(timer.stop())
    casosc = sorted_casosc.copy()
    casost = sorted_casost.copy()

    casosc.to_excel('casosc.xlsx')
    casost.to_excel('casost.xlsx')
    exit()

else:
    casosc['identificacao'] = casosc['identificacao'].apply(int)
    casost['identificacao'] = casost['identificacao'].apply(int)
    casosc = casosc.sort_values('identificacao').copy()
    casost = casost.sort_values('identificacao').copy()

# casosc['identificacao'] = casost['identificacao'].copy()

In [13]:
print(all(casosc.sort_values('nome_hash')['nome_hash'].values == casost.sort_values('nome_hash')['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
print(all(casosc['idade'].values == casost['idade'].values))

True
True
False


In [14]:
obitosc = casosc.loc[casosc['obito']=='SIM']
obitost = casost.loc[casost['obito']=='SIM']

In [15]:
obitosc.loc[~obitosc['identificacao'].isin(obitost['identificacao'])]

,ordem,identificacao,ibge7,rs_res_pr,nome,sexo,idade,mun_resid,mun_atend,rs,laboratorio,dt_diag,comunicacao,is,obito,data_obito,excluir,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,hash_dt_diag,nome_hash


In [16]:
obitost.loc[~obitost['identificacao'].isin(obitosc['identificacao'])]

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,hash,hash_less,hash_more,hash_atend,hash_less_atend,hash_more_atend,nome_hash


In [17]:
regionais = static.regionais.copy()

try:
    casosc = casosc_copy.copy()
except:
    casosc_copy = casosc.copy()

casosc['ordem'] = [ i for i in range(1,len(casosc)+1)]

casosc = casosc.rename(columns={'ibge7':'ibge_resid','nome':'paciente','dt_diag':'data_diagnostico','comunicacao':'data_comunicacao','is':'data_1o_sintomas'})

casosc['ibge_atend'] = casost['ibge_atend_pr'].apply(normalize_ibge).values.tolist()
casosc['ibge_resid'] = casosc['ibge_resid'].apply(normalize_ibge).values.tolist()

casosc.loc[casosc['rs'].isnull(),'rs'] = 99
casosc['rs'] = casosc['rs'].apply(int)

regionais = regionais.append(pd.Series({'nu_reg':99,'nm_macro':"FORA_PR",'nm_reg':'FORA_PR'}),ignore_index=True)
regionais = regionais.groupby(['nu_reg','nm_reg','nm_macro'])[['ibge']].count().reset_index()
# del regionais['ibge']
# casosc['nm_macro'] = casosc.apply(lambda x: regionais.loc[regionais['nu_reg']==x['rs']],1)
# casosc['nm_reg'] = casosc.apply(lambda x: regionais.loc[regionais['nu_reg']==x['rs']],1)
casosc = pd.merge(casosc,regionais,how='left',left_on='rs',right_on='nu_reg',left_index=True)


casosc = pd.merge(casosc,casost[['identificacao','dt_obito','dt_com_obito','status','dt_com_recuperado']],'left',on='identificacao')

casosc['caso'] = 1
casosc['recuperado'] = 0
casosc['ativo'] = 1

casosc.loc[(casosc['status']=="Recuperado"),'recuperado'] = 1
casosc.loc[(casosc['obito']=='SIM'),'obito'] = 1
casosc.loc[(casosc['obito']=='NAO'),'obito'] = 0
casosc.loc[(casosc['obito']==1) | (casosc['recuperado']==1),'ativo'] = 0

casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'].to_excel('casos_cura_obito.xlsx')
casosc.loc[(casosc['recuperado']==1) & (casosc['obito']==1 ), 'recuperado'] = 0

casosc['data_com_evolucao'] = ''

casosc.loc[casosc['obito']==1,'evolucao'] = 'OBITO'
casosc.loc[casosc['recuperado']==1,'evolucao'] = 'CURA'
casosc.loc[casosc['ativo']==1,'evolucao'] = 'ATIVO'

casosc = casosc.rename(columns={'data_obito':'data_evolucao'})

casosc = casosc[['nm_macro','ordem', 'identificacao', 'ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'rs','nm_reg', 'mun_resid', 'mun_atend', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao','data_evolucao', 'data_com_evolucao']]

In [18]:
a = casosc.groupby('evolucao')[['ordem']].count().rename(columns={'ordem':'qtde'})
a.append(pd.DataFrame([len(casosc)],['TOTAL CASOS'],['qtde']))

,qtde
ATIVO,229868
CURA,587794
OBITO,16124
TOTAL CASOS,833786


In [19]:
totais_macro = casosc.groupby(['nm_macro','evolucao'])[['ordem']].count()
totais_macro = totais_macro.unstack().droplevel(0,1)
totais_macro['TOTAL_CASOS'] = totais_macro.sum(1)
totais_macro = totais_macro.append(pd.Series(totais_macro.sum(0),name='TOTAL_GERAL'))
totais_macro = totais_macro.reindex(['LESTE', 'OESTE', 'NOROESTE', 'NORTE', 'FORA_PR', 'TOTAL_GERAL'])
totais_macro

evolucao,ATIVO,CURA,OBITO,TOTAL_CASOS
nm_macro,,,,
LESTE,107025,271484,8080,386589
OESTE,25273,149255,2931,177459
NOROESTE,34826,93672,2235,130733
NORTE,59457,71407,2761,133625
FORA_PR,3287,1976,117,5380
TOTAL_GERAL,229868,587794,16124,833786


In [20]:
totais_rs = casosc.groupby(['rs','nm_reg','evolucao'])[['ordem']].count()
totais_rs = totais_rs.unstack().droplevel(0,1)
totais_rs['TOTAL_CASOS'] = totais_rs.sum(1)
totais_rs = totais_rs.append(pd.Series(totais_rs.sum(0),name=('','TOTAL_GERAL')))
totais_rs

,evolucao,ATIVO,CURA,OBITO,TOTAL_CASOS
rs,nm_reg,,,,
1,PARANAGUA,10556,14323,532,25411
2,METROPOLITANA,62636,194435,5855,262926
3,PONTA GROSSA,15133,29542,855,45530
4,IRATI,1345,6927,142,8414
5,GUARAPUAVA,8512,13431,323,22266
6,UNIAO DA VITORIA,1474,4698,100,6272
7,PATO BRANCO,3115,17719,400,21234
8,FRANCISCO BELTRAO,6921,21943,428,29292
9,FOZ DO IGUACU,8706,36125,774,45605


In [21]:
aux = casosc.copy()
aux.loc[aux['rs']==99,'mun_resid'] = 'FORA_PARANA'
totais_mun = aux.groupby(['ibge_resid','mun_resid','evolucao'])[['ordem']].count()
totais_mun = totais_mun.unstack().droplevel(0,1)
totais_mun['TOTAL_CASOS'] = totais_mun.sum(1)
totais_mun.append(pd.Series(totais_mun.sum(0),name='TOTAL_GERAL'))
totais_mun = totais_mun.fillna(0).astype(int)
totais_mun

,evolucao,ATIVO,CURA,OBITO,TOTAL_CASOS
ibge_resid,mun_resid,,,,
410010,ABATIA,43,305,6,354
410020,ADRIANOPOLIS,107,868,20,995
410030,AGUDOS DO SUL,143,260,5,408
410040,ALMIRANTE TAMANDARE,5299,1356,220,6875
410045,ALTAMIRA DO PARANA,19,71,2,92
...,...,...,...,...,...
412863,DOUTOR ULYSSES,16,197,2,215
412865,VIRMOND,167,13,3,183
412870,VITORINO,30,474,14,518


In [22]:
writer = pd.ExcelWriter(join(output,"totais_pr.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

totais_macro.to_excel(writer,'totais_macro')
worksheet = writer.sheets['totais_macro']
auto_fit_columns(worksheet,totais_macro)

totais_rs.to_excel(writer,'totais_rs')
worksheet = writer.sheets['totais_rs']
auto_fit_columns(worksheet,totais_rs)

totais_mun.to_excel(writer,'totais_mun')
worksheet = writer.sheets['totais_mun']
auto_fit_columns(worksheet,totais_mun)

writer.save()

In [23]:
writer = pd.ExcelWriter(join(output,"casos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc['id'] = [ f"{rs}{i}" for i,rs in enumerate(casosc['nm_macro'])]
casosc['id_notifica'] = ''

casosc = casosc[['id','identificacao','id_notifica','ibge_resid', 'ibge_atend', 'paciente', 'sexo', 'idade', 'rs', 'mun_resid', 'mun_atend', 'laboratorio', 'data_diagnostico', 'data_comunicacao', 'data_1o_sintomas', 'evolucao', 'data_evolucao', 'data_com_evolucao']].sort_values(['data_comunicacao','paciente'])
casosc.to_excel(writer,'casos_confirmados',index=None)
worksheet = writer.sheets['casos_confirmados']
auto_fit_columns(worksheet,casosc)

writer.save()

In [24]:
obitos = casosc.loc[casosc['evolucao']=='OBITO']
writer = pd.ExcelWriter(join(output,"obitos_confirmados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

obitos = obitos[['id','identificacao','id_notifica','ibge_resid', 'paciente', 'sexo', 'idade', 'rs', 'mun_resid', 'data_diagnostico', 'data_evolucao', 'data_com_evolucao']].sort_values(['data_evolucao','paciente'])
obitos.to_excel(writer,'obitos_confirmados',index=None)
worksheet = writer.sheets['obitos_confirmados']
auto_fit_columns(worksheet,obitos)

writer.save()

In [25]:
recuperados = casosc.loc[casosc['evolucao']=='CURA']
writer = pd.ExcelWriter(join(output,"casos_recuperados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

recuperados = recuperados[['id','identificacao','id_notifica','ibge_resid', 'paciente', 'sexo', 'idade', 'rs', 'mun_resid', 'data_diagnostico', 'data_evolucao', 'data_com_evolucao']].sort_values(['data_diagnostico','paciente'])
recuperados.to_excel(writer,'casos_recuperados',index=None)
worksheet = writer.sheets['casos_recuperados']
auto_fit_columns(worksheet,recuperados)

writer.save()

In [26]:
ativos = casosc.loc[casosc['evolucao']=='ATIVO'].copy()
ativos['dias_apos_notificacao'] = ativos.apply(lambda row: (pd.Timestamp('today') - row['data_diagnostico']).days
, axis=1)

writer = pd.ExcelWriter(join(output,"casos_ativos.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

ativos = ativos[['id','identificacao','id_notifica','ibge_resid', 'paciente', 'sexo', 'idade', 'rs', 'mun_resid', 'data_diagnostico', 'data_comunicacao','dias_apos_notificacao']].sort_values(['dias_apos_notificacao','paciente'],ascending=False)
ativos.to_excel(writer,'casos_ativos',index=None)
worksheet = writer.sheets['casos_ativos']
auto_fit_columns(worksheet,ativos)

writer.save()

In [27]:
# my_red = openpyxl.styles.colors.Color(rgb='00FF0000')
# my_fill = openpyxl.styles.fills.PatternFill(patternType='solid', fgColor=my_red)
# cell.fill = my_fill